# Feature selection.

We use feature selection after wrangling for two reasons.

1. Obtain a set of good features that represents the current dataset.
2. Obtain the set of *not good* features that should be refined in the next wrangling step.

The following methods are implemented in this notebook.

1. A (baseline) random sampling based approach.
2. CHCGA — a genetic algorithm based approach.
3. ASFSS — a forward selection based approach.
4. AdaBoost with decision stump approach.

Both (1) and (2) allow use to set a max running time.

In [34]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from tqdm.notebook import tqdm
from avatar.language import WranglingLanguage
from avatar.analysis import *

In [35]:
titanic = pd.read_csv("../data/raw/demo/titanic.csv")
display(titanic.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
language = WranglingLanguage()
expanded = language.expand(titanic, target="Survived")
expanded.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,NaN(111240)(Ticket)_Ticket,NaN(111428)(Ticket)_Ticket,NaN(111361)(Ticket)_Ticket,NaN(111320)(Ticket)_Ticket,NaN(111427)(Ticket)_Ticket,NaN(111426)(Ticket)_Ticket,NaN(19996)(Ticket)_Ticket,NaN(111369)(Ticket)_Ticket,NaN(C111)(Cabin)_Cabin,WordToNumber()(Ticket)_Ticket
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,C85,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,113803,113803,113803,113803,113803,113803,113803,113803,C123,113803.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,373450,373450,373450,373450,373450,373450,373450,373450,NaN,373450.0


We sample a subset of the data with at least one row containing no NaNs.

In [37]:
sampler = WeightedColumnSampler(expanded)
sampled = sampler.sample()
sampled

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Split(-)(Name)_0,...,NaN(C.A. 33111)(Ticket)_Ticket,NaN(371110)(Ticket)_Ticket,NaN(111240)(Ticket)_Ticket,NaN(111428)(Ticket)_Ticket,NaN(111361)(Ticket)_Ticket,NaN(111320)(Ticket)_Ticket,NaN(111427)(Ticket)_Ticket,NaN(111426)(Ticket)_Ticket,NaN(19996)(Ticket)_Ticket,NaN(111369)(Ticket)_Ticket
0,1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,"Braund, Mr. Owen Harris",...,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171,A/5 21171
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",...,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599,PC 17599
2,3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,"Heikkinen, Miss. Laina",...,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282,STON/O2. 3101282
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",...,113803,113803,113803,113803,113803,113803,113803,113803,113803,113803
4,5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,"Allen, Mr. William Henry",...,373450,373450,373450,373450,373450,373450,373450,373450,373450,373450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,0,0,211536,13.0000,"Montvila, Rev. Juozas",...,211536,211536,211536,211536,211536,211536,211536,211536,211536,211536
887,888,1,1,"Graham, Miss. Margaret Edith",female,0,0,112053,30.0000,"Graham, Miss. Margaret Edith",...,112053,112053,112053,112053,112053,112053,112053,112053,112053,112053
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,1,2,W./C. 6607,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",...,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607,W./C. 6607
889,890,1,1,"Behr, Mr. Karl Howell",male,0,0,111369,30.0000,"Behr, Mr. Karl Howell",...,111369,111369,111369,111369,111369,111369,111369,111369,111369,NaN


## Preselection

The first step is a preprocessing step aimed at removing highly similar features. A wrapper approach is used, in which we compare the performance of decision stumps.

In [38]:
_mercs_stump = dict(
    # Induction
    max_depth=1,
    selection_algorithm="default",
    nb_targets=1,
    nb_iterations=1,
    n_jobs=1,
    # Inference
    inference_algorithm="own",
    prediction_algorithm="mi",
    max_steps=8,
)

Remove selected feature at each step.

In [39]:
# def select_greedy(df, target):
    
#     data, nominal = to_mercs(df)
#     custom_m_code = to_m_codes(df, target)
#     custom_q_code = custom_m_code[0].astype(int)

#     predictions = list()
    
#     for i in tqdm(range(len(df.columns) - 1)):
# #     for i in range(len(df.columns)):
      
#         # build model
#         model = Mercs(**_mercs_stump)
#         model.fit(data, nominal_attributes=nominal, m_codes=custom_m_code)

#         # get predictions
#         predictions.append(model.predict(np.nan_to_num(data), q_code=custom_q_code))
        
#         # get used feature
#         f = np.where(model.m_fimps)[1][0]
#         custom_m_code[:, f] = -1
#         custom_q_code[f] = -1
        
# #         print("---- iteration {} -----".format(i))
# #         print(custom_m_code)
# #         print(custom_q_code)
            
# #         if i == 5:
# #             break

# select_greedy(sampled, "Survived")

In [40]:
# def select_iteratively(df, target):
    
#     data, nominal = to_mercs(df)
    
#     # make m code and hide everything
#     custom_m_code = to_m_codes(df, target)
#     custom_m_code[custom_m_code == 0] = -1
    
#     # make q code and hide everything
#     custom_q_code = custom_m_code[0].astype(int)
#     custom_q_code[custom_q_code == 0] = -1

#     target_i = np.where(custom_q_code == 1)[0][0]

#     predictions = list()
    
#     for i in tqdm(range(len(df.columns))):
        
#         if i == target_i:
#             continue
        
#         new_m_code = np.copy(custom_m_code)
#         new_m_code[:, i] = 0
#         new_q_code = np.copy(custom_q_code)
#         new_q_code[i] = 0
       
#         # build model
#         model = Mercs(**_mercs_stump)
#         model.fit(data, nominal_attributes=nominal, m_codes=new_m_code)

#         # get predictions
#         predictions.append(model.predict(np.nan_to_num(data), q_code=new_q_code))
        
#         # get used feature
#         f = np.where(model.m_fimps)[1][0]
#         custom_m_code[:, f] = -1
#         custom_q_code[f] = -1


# select_iteratively(sampled, "Survived")

## Evaluation

How do we evaluate a set of features?

* Which **model** do we use?
* what **max depth** do we use?

### Timing

Small experiment on runtime for decision trees of different depths.

In [51]:
import time
import itertools


classifiers = ["DT", "XGB"]
depths = [4, 8, 16, 32]
iterations = 1

data, nominal = to_mercs(sampled)
custom_m_code = to_m_codes(sampled, "Survived")


results = list()
for classifier, depth in tqdm(list(itertools.product(classifiers, depths))):
    start = time.time()
#     scores = list()
    for i in range(iterations):
        model = Mercs(classifier_algorithm=classifier,
                      max_depth=depth,
                      nb_targets=1,
                      random_state=i)
        model.fit(data, nominal_attributes=nominal, m_codes=custom_m_code)
#         scores.append(model.m_score)
#     results.append((classifier, depth, time.time() - start, np.mean(scores)))
    results.append((classifier, depth, time.time() - start))

# for classifier, depth, t, score in results: 
#     print("{}\t{}\t{}\t{}".format(classifier, depth, t, score))
for classifier, depth, t in results: 
    print("{}\t{}\t{}".format(classifier, depth, t))


DT	4	0.11190295219421387
DT	8	0.11357712745666504
DT	16	0.10427975654602051
DT	32	0.10384225845336914
XGB	4	0.10079407691955566
XGB	8	0.1020650863647461
XGB	16	0.11010122299194336
XGB	32	0.20048809051513672


Next, we can take a look at actual feature selection. Three methods are implemented.

* Classic sequential and backwards sequential feature selection.
* Randomly sampling columns, training a model and getting the feature relevances.
* A genetic approach, which is similar but should combine features slightly better. We perform a small experiment on whether to fix the genome size.
* AdaBoost with decision stumps.

In [42]:
class Selector:
    
    def __init__(self, df: pd.DataFrame):
        self._df = df
        self._importances = np.zeros_like(df.columns, dtype=float)
    
    def ordered(self):
        return np.argsort(self._importances)
    
    def select(self, n: int):
        return self._df.columns[self.ordered()[:n]]

### Random

Randomly sample subsets of features, evaluate and get feature relevance. Weight relevances over different iterations.

In [43]:
def sample_features(df: pd.DataFrame, max_features: int):
    """Sample some features."""
    pass


class SamplingSelector(Selector):
    pass

### Genetic

Constrained genetic algorithm.

In [44]:
import random


class Individual: 
    
    def __init__(self, genome):
        self._genome = genome
    
    def cross(self, other: "Individual", threshold: int):
        """Perform Half-Uniform Crossover (HUX)."""
        assert threshold < len(self.genome)
        assert len(self.genome) == len(other.genome)
        # get locations where differ
        (different,) = np.where(self.genome != other.genome)
        # don't cross over
        if len(different) // 2 < threshold:
            return None
        # select chromosomes to swamp
        indices = np.random.choice(different, len(different) // 2, replace=False)
        # make children
        c1 = np.copy(self.genome)
        c1[indices] = other.genome[indices]
        c2 = np.copy(other.genome)
        c2[indices] = self.genome[indices]
        return Individual(c1), Individual(c2)
    
    def mutate(self, p: float) -> "Individual":
        """Mutation."""
        assert p < 1
        indices = np.random.choice(
            np.arange(len(self.genome)),
            int(p * len(self.genome)),
            replace=False
        )
        genome = np.copy(self.genome)
        genome[indices] = 1 - genome[indices]
        return Individual(genome)
        
    @property
    def genome(self):
        return self._genome
    
    @classmethod
    def generate(cls, n: int) -> "Individual":
        """Generate random genome."""
        return Individual(np.random.randint(2, size=n))

    @classmethod
    def generate_for(cls, df: pd.DataFrame) -> "Individual":
        return cls.generate(df.shape[1])
    
    def __str__(self):
        return str(self._genome)
    
    def __display__(self):
        return str(self)

    
def evaluate(population, df, target):
    """Evaluate all individuals in a population."""
    data, nominal = to_mercs(df)
    code = to_m_codes(df, target)
    return [evaluate_single(individual, (data, nominal, code)) for individual in population]


def evaluate_single(individual, target):
    """Evaluate a single individual."""
    data, nominal, m_code = target
    model = Mercs()
    model.fit(data, nominal_attributes=nominal, m_codes=m_code)
    return model.m_score


i1 = Individual.generate_for(sampled)
i2 = Individual.generate_for(sampled)
# evaluate([i], sampled, "Survived")

In [45]:
def evaluate(population, df, target):
    """Evaluate all individuals in a population."""
    data, nominal = to_mercs(df)
    code = to_m_codes(df, target)
    return [evaluate_single(individual, (data, nominal, code)) for individual in population]


def evaluate_single(individual, target):
    """Evaluate a single individual."""
    data, nominal, m_code = target
    model = Mercs()
    model.fit(data, nominal_attributes=nominal, m_codes=m_code)
    return model.m_score


i = Individual.generate(sampled.shape[1])
evaluate([i], sampled, "Survived")

[array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

### ASFSS

### AdaBoost